In [1]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1JOKToaq1PSyQMK9Q2abDejqlzmKaiGyNH0tzwx3kCds/export?format=csv'

r = requests.get(google_sheet_url)
data = r.content
df = pd.read_csv(BytesIO(data))
local_index = []
df.head(5)





,Sl. NO,Agent Code,First Name,Last Name,State,District,Taluk,Hobli,Village,Pincode,...,Cheque No,Bank Name,Branch Name,Bank Account Name,Account No,IFSC Code,PAN Card Number,Area,Agreement Signature,备注\nRemark
0,13,AG014,Kumaraswamy,Muddappa,Karnataka,Chamarajanagar,Gundlupet,Begur,Aralikatte,571109,...,933685,Canara Bank,Mukkadahalli,Kumaraswamy M,5432101000919,CNRB0005432,DDRPM1650R,1.Gundlupet 2. Tamilnadu,Singnature,NaN
1,6,AG012,Chandrashekhararadya,Channaveeraradya,Karnataka,Mysuru,Nanjangud,Hullahalli,Ballurundi,111111,...,611135,Vijaya Bank,"Santhepete, Mysore",Chandrashekhararadya C,119101011002689,VIJB0001191,NaN,Nanjangud,Singnature,Pan Card Copy not Received
2,7,AG013,Gopala,Hanumaiah,Karnataka,Mysuru,Nanjangud,Hullahalli,Bankahalli,571121,...,121700,Canara Bank,Gundlupet,Gopala H,5433101001234,CNRB0004792,CQNPG6319K,Sargur,Singnature,IFSC Code and Branch name change 15-06-2019
3,11,AG041,Basappa,Kadappa,Karnataka,Chamarajanagar,Gundlupet,Gundlupet,Bargi,571111,...,608580,State Bank of India,Gundlupet,Basappa K,30442134675,SBIN0010364,BZIPB5090H,Gundlupet,Singnature,NaN
4,24,AG042,Maheshappa,Basappa,Karnataka,Chamarajanagar,Gundlupet,Gundlupet,Bargi,571111,...,315506,State Bank of India,Gundlupet,Maheshappa,30447721034,SBIN0010364,DCZPM4497M,Gundlupet,Singnature,NaN


In [2]:
def isNaN(num):
    return num != num

In [3]:
# Uploading agents int User, UserProfile models

busines = Business.objects.get(id=1)

count = 0
for index, row in df.iterrows():
    count = count+1
    
    print('{}:{}'.format(row['First Name'], row['Agent Code']))
    
    
    user, created = User.objects.get_or_create(
        username=str(row['Contact No']),
        first_name=row['First Name'],
        last_name=row['Last Name'],
        defaults = {
            'is_staff': True,
            'is_active': True,
            'is_superuser': False,
            'password':make_password(str(row['Contact No'])[-4:])
            
        }
    )
    if created:
        print(user.username, 'Count:', count, "\tUser created")
    else:
        print(user.username, 'Count:', count, "\tUser existed already")
        
        
    
    # Add new hobli if not available already
    if not Hobli.objects.filter(name=row['Hobli']).exists():
        hobli, created = Hobli.objects.get_or_create(
            name=str(row['Hobli']),
            taluk=Taluk.objects.get(name=row['Taluk']),
            defaults = {
            }
        )
        if created:
            print("\t", hobli.name, "\tHobli created")
        else:
            print("\t", hobli.name, 'Count:', "\tHobli existed already")
    
    village_name=row['Village'].rstrip(' ')
    village, created = Village.objects.get_or_create(
            name=str(village_name),
            hobli=Hobli.objects.get(name=row['Hobli']),
            defaults = {
            }
    )
    if created:
        print("\t", village.name, "\tVillage created")
    else:
        print("\t", village.name, "\tVillage existed already")
        
        
            
        
    
    user_profile, created = UserProfile.objects.update_or_create(
        user=user,
        business=Business.objects.get(id=1),
        defaults = {
            'code':row['Agent Code'],
            'user_type': UserType.objects.get(name='Agent'),
            'gender': Gender.objects.get(name='Male'),
            # 'dob':,
            'mobile':row['Contact No'],
            # 'alternate_mobile':,
            # 'email':,
            'address':row['Address'],
            'state':State.objects.get(name=row['State']),
            'district':District.objects.get(name=row['District']),
            'taluk':Taluk.objects.get(name=row['Taluk']),
            'hobli':Hobli.objects.get(name=row['Hobli']),
            'village':village,
            'pincode':row['Pincode'],
            #'educational_qualification':,
            #'institution_name':row[''],
            #'university_name':row[''],
            'aadhaar_number':row['ID Prof NO'],
            #'aadhaar_document':row[''],
            'pan_number':row['PAN Card Number'],
            #'pan_document':row[''],
            #'driving_licence_document':row[''],
            #'agreement_document':row[''],
            #'prior_experience_in_other_company':row[''],
            #'prior_experience_company_name':row[''],
            #'prior_experience_duration':row[''],
            #'collaborated_with_company_on':row[''],
            
            #'latitude':row[''],
            #'longitude':row[''],
            'created_by':User.objects.get(id=13),
            'modified_by':User.objects.get(id=13)
        }
    )
    if created:
        print("\t\tUser Profile Created")
    else:
        print("\t\t\tUser Profile existed already:" + str(user_profile.id))
        
    
    # Add bank details for Agent
    user_bank, created = UserBankDetails.objects.get_or_create(
        user=user,
        bank=row['Bank Name'],
        branch=row['Branch Name'],
        ifsc_code=row['IFSC Code'],
        account_holder_name=row['Bank Account Name'],
        account_number=row['Account No'],
        post_cheque_number=row['Cheque No'],
        defaults = {
            'created_by':User.objects.get(id=13),
            'modified_by':User.objects.get(id=13)    
        }
    )
    if created:
        print("\t\t\tUser Bank  Created")
    else:
        print("\t\t\tUser Bank existed already")
      
    
    # User Office info for agent
    agent_office, created = AgentOffice.objects.update_or_create(
        user=user,
        business=Business.objects.get(id=1),
        name='Office of' + row['First Name'],
        mobile=row['Contact No'],
        pincode=row['Pincode'],
        defaults = {
            'email':'sample@gmail.com',
            'address':row['Address'],
            'state':State.objects.get(name=row['State']),
            'district':District.objects.get(name=row['District']),
            'taluk':Taluk.objects.get(name=row['Taluk']),
            'hobli':Hobli.objects.get(name=row['Hobli']),
            'village':village,
            'created_by':User.objects.get(id=13),
            'modified_by':User.objects.get(id=13)    
        }
    )
    if created:
        print("\t\t\tUser Office  Created")
    else:
        print("\t\t\tUser Office existed already")
        
    
 

Kumaraswamy:AG014
9880838227 Count: 1 	User existed already
	 Aralikatte 	Village existed already
			User Profile existed already:45
			User Bank existed already
			User Office existed already
Chandrashekhararadya:AG012
9632820922 Count: 2 	User existed already
	 Ballurundi 	Village existed already
			User Profile existed already:43
			User Bank existed already
			User Office existed already
Gopala:AG013
7406990508 Count: 3 	User existed already
	 Bankahalli 	Village existed already
			User Profile existed already:44
			User Bank existed already
			User Office existed already
Basappa:AG041
9632034111 Count: 4 	User existed already
	 Bargi 	Village existed already
			User Profile existed already:72
			User Bank existed already
			User Office existed already
Maheshappa:AG042
8971034363 Count: 5 	User existed already
	 Bargi 	Village existed already
			User Profile existed already:73
			User Bank existed already
			User Office existed already
Nagappa:AG043
9663958671 Count: 6 	User existe

			User Bank existed already
			User Office existed already


In [3]:
busines = Business.objects.get(id=1)

In [5]:
busines

<Business: 1-ccgb>

In [8]:
'rerere   '.rstrip(' ')

'rerere'